# High Performance Pandas

```py
mask = (x > .5) & (y < .5)
df[df[mask]]
```

under the hood:
```py
tmp1 = (x > .5)
tmp2 = (y < .5)
mask = tmp1 & tmp2
```

use pd.eval(""), df.query()-> does element-wise operation with numexpr

```py
```

In [2]:
import numpy as np
import pandas as pd

nrows, ncols = 1_000_000, 100

df1, df2, df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.376684,0.622524,1.115075,0.817168,0.778488,-0.911294,-0.252389,-0.248771,-0.115617,-0.389029,...,0.966055,-1.110027,-0.145951,0.611313,-1.112700,0.154077,0.403221,-1.032422,-0.466617,0.082386
1,-1.262694,0.663337,-0.582510,0.886458,-0.956660,0.255060,0.431877,0.768787,0.169523,-1.359059,...,0.353395,0.160541,-0.730282,-1.093096,0.720355,0.367616,-2.217241,-1.268865,1.198168,1.622763
2,-1.225309,-0.648595,-0.137212,-0.036743,1.790903,0.716509,-0.430757,0.336094,-1.972866,-0.344478,...,-0.052118,0.922231,-0.184486,1.427414,-0.029566,-1.122952,-0.560718,2.184769,-1.020319,0.578783
3,-0.038274,0.288647,0.062988,0.327359,-0.033139,0.535108,-1.222301,-0.733129,-1.980211,-0.965344,...,0.093781,0.078589,0.604998,-0.119185,-0.777408,0.566290,-0.858216,0.967794,0.269115,-1.472879
4,-0.354281,0.045646,-0.642474,1.908437,-0.714575,0.482162,2.373619,-1.398995,-0.747867,0.421300,...,-1.200618,0.167278,-0.719155,0.923082,-0.696651,-0.041987,0.424188,0.732535,-1.402559,1.616225


In [4]:
df1.shape

(1000000, 100)

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 100 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   0       1000000 non-null  float64
 1   1       1000000 non-null  float64
 2   2       1000000 non-null  float64
 3   3       1000000 non-null  float64
 4   4       1000000 non-null  float64
 5   5       1000000 non-null  float64
 6   6       1000000 non-null  float64
 7   7       1000000 non-null  float64
 8   8       1000000 non-null  float64
 9   9       1000000 non-null  float64
 10  10      1000000 non-null  float64
 11  11      1000000 non-null  float64
 12  12      1000000 non-null  float64
 13  13      1000000 non-null  float64
 14  14      1000000 non-null  float64
 15  15      1000000 non-null  float64
 16  16      1000000 non-null  float64
 17  17      1000000 non-null  float64
 18  18      1000000 non-null  float64
 19  19      1000000 non-null  float64
 20  20      1000000 non-null

In [7]:
%timeit df1+df2+df3+df4
%timeit pd.eval("df1+df2+df3+df4")

457 ms ± 4.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
196 ms ± 3.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
standard = df1+df2+df3+df4
sum_eval = pd.eval("df1+df2+df3+df4")

In [8]:
rolls = pd.DataFrame(np.random.randint(1,6, (6,3)), columns = ["Die1", "Die2", "Die3"])
rolls

,Die1,Die2,Die3
0,4,1,4
1,4,4,2
2,2,3,5
3,5,5,4
4,3,4,2
5,4,4,5


In [13]:
rolls.eval("Sum = Die1+Die2+Die3", inplace = True)
rolls

,Die1,Die2,Die3,Sum
0,4,1,4,9
1,4,4,2,10
2,2,3,5,10
3,5,5,4,14
4,3,4,2,9
5,4,4,5,13


In [19]:
high = 11
rolls.eval("Winner = Sum > @high")

,Die1,Die2,Die3,Sum,Winner
0,4,1,4,9,False
1,4,4,2,10,False
2,2,3,5,10,False
3,5,5,4,14,True
4,3,4,2,9,False
5,4,4,5,13,True


In [20]:
# traditional way of filtering
rolls[rolls["Sum"] > high]

,Die1,Die2,Die3,Sum,Winner
3,5,5,4,14,True
5,4,4,5,13,True


## Query

- filter using query

In [24]:
rolls.query("Sum > @high")

,Die1,Die2,Die3,Sum,Winner
3,5,5,4,14,True
5,4,4,5,13,True


In [25]:
rolls.query("Sum > @high & Die1 == 4")

,Die1,Die2,Die3,Sum,Winner
5,4,4,5,13,True


In [ ]:
# NOTE when optimisation is not needed, keep readability in mind firstly, using query for example can make it less readable

In [26]:
df_os = pd.read_csv("../Data/athlete_events.csv")
df_os

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [27]:
# get "View Raw" URL from GitHub
link = "https://raw.githubusercontent.com/Andreas-Svensson/Databehandling-Andreas-Svensson/main/Data/athlete_events.csv"
df_os = pd.read_csv(link)
df_os

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [40]:
%timeit df_os[df_os["NOC"] == "SWE"].head()
# for query need to use quotes around SWE since we are looking for a value rather than a column
%timeit df_os.query("NOC == 'SWE'").head()

15.4 ms ± 146 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
8.17 ms ± 81.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [41]:
# NOTE in this example its SLOWER to use query, it is not always faster
# generally it is faster when masking on multiple conditions, because of reducing the intermediates
%timeit df_os[df_os["Height"] > 180]
%timeit df_os.query("Height > 180")

9.1 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.3 ms ± 799 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
%timeit df_os[(df_os["Sex"] == "F") & (df_os["Height"] > 180) & (df_os["NOC"] == "SWE")]
%timeit df_os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'")

27.9 ms ± 369 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
10.5 ms ± 165 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
